In [1]:
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [2]:
# AttentionWithContext adapted for Tensorflow 1.13 with Eager Execution.
# IMPORTANT -you can't use regular keras optimizers.  You need to grab one that is subclassed from 
# tf.train.Optimizer.   Not to worry, your favorite is probably there, for example - 
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer

# That's it, now you can use this layer - 
# Adapted from https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2

# Tested using functional API.   Just plop on top of an RNN, like so - 

# x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], input_length=max_topic_length, trainable=False)(inputs)
# x1 = LSTM(return_sequences=True)(x)
# c1 = AttentionWithContext()(x1)

import tensorflow as tf
from tensorflow.keras import initializers 
from tensorflow.keras import regularizers
from tensorflow.keras import constraints

from tensorflow.keras import activations 
from tensorflow.keras import backend as K

from tensorflow.keras.layers import Layer

class AttentionWithContext(Layer):
    """
        Attention operation, with a context/query vector, for temporal data.
        Supports Masking.
        Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
        "Hierarchical Attention Networks for Document Classification"
        by using a context vector to assist the attention
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(AttentionWithContext())
        """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True,
                 return_attention=False, **kwargs):

        self.supports_masking = True
        self.return_attention = return_attention
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        input_shape_list = input_shape.as_list()

        self.W = self.add_weight(shape=((input_shape_list[-1], input_shape_list[-1])),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape_list[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape_list[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape.as_list())

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = tf.tensordot(x, self.W,axes=1)

        if self.bias:
            uit += self.b

        uit = activations.tanh(uit)
        # ait = K.dot(uit, self.u)
        ait = tf.tensordot(uit, self.u,axes=1)

        a = activations.exponential(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= tf.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= tf.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        result = K.sum(weighted_input, axis=1)

        if self.return_attention:
            return [result, a]
        return result

    def compute_output_shape(self, input_shape):
        if self.return_attention:
            #TODO use TensorShape here, as done in the else statement.   I'm not sure
            # if this is returning a single tensor, or a list of two so leaving this undone for now.  Suspect this will
            # need to complete if using Sequential rather than Functional API
            return [(input_shape[0], input_shape[-1]),
                    (input_shape[0], input_shape[1])]
        else:
            return tf.TensorShape([input_shape[0].value,input_shape[-1].value])

        
    def get_config(self):
        config = {
            'W_regularizer': initializers.serialize(self.W_regularizer),
            'u_regularizer': initializers.serialize(self.u_regularizer),
            'b_regularizer': initializers.serialize(self.b_regularizer),

            'W_constraint': constraints.serialize(self.W_constraint),
            'u_constraint': constraints.serialize(self.u_constraint),
            'b_constraint': constraints.serialize(self.b_constraint),

            'return_attention': self.return_attention,

            'bias': self.bias,
        }

        base_config = super(AttentionWithContext, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [3]:
def load_models():
    categorizing_model = load_model("./best_model_glove_attention_200d.h5", custom_objects={'AttentionWithContext': AttentionWithContext})
    filtering_model = load_model("../Tweets-Filtering/bilstm_model.h5", compile=False)

    return categorizing_model, filtering_model


def load_tokenizers():
    categorizing_model_tokenizer = pickle.load(open("./tokenizer.pkl", "rb"))
    filtering_model_tokenizer = pickle.load(open("../Tweets-Filtering/tokenizer.pkl", "rb"))

    return categorizing_model_tokenizer, filtering_model_tokenizer


def createDataFrame(tweets_list):
    tweets_df = pd.DataFrame(data=tweets_list, columns=[
        "tweet_id", "tweet_text"])

    return tweets_df


# Identify locations before alof of preprocessing of the tweets
def identify_locations(tweets_list):
    pass


def predict_filter_relevance(filtering_model, filtering_tokenizer, tweets_text):
    # TOKENIZE TWEETS
    sequences = filtering_tokenizer.texts_to_sequences(tweets_text)
    max_sequence_len = 50
    tweets_text_padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len)

    relevance_preds = filtering_model.predict(tweets_text_padded_sequences)
    relevance_preds_cat = np.argmax(relevance_preds, axis=1)

    return relevance_preds_cat


def mapCategories(category):
    category_labels_dict = {0: 'Sympathy and Support',
         1: 'Caution and Advice',
         2: 'Relief Request and Donations',
         3: 'Damage Report',
         4: 'Casualty'}

    return category_labels_dict[category]


def predict_categories(categorizing_model, categorizing_tokenizer, tweets_text):
    # TOKENIZE TWEETS
    sequences = categorizing_tokenizer.texts_to_sequences(tweets_text)
    max_sequence_len = 35
    tweets_text_padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='post')

    category_preds = categorizing_model.predict(tweets_text_padded_sequences)
    print(category_preds)
    category_preds = np.argmax(category_preds, axis=1)
    print()
    print(relevance_preds_cat)

    vect_mapCategories = np.vectorize(mapCategories)

    return vect_mapCategories(category_preds)

In [4]:
test_data = pd.read_csv("../Web App/api/tweets/Tweets with Locations_Nepal Earthquake.csv")

# test_data = pd.read_csv("C:/Users/Kelvin Chumbe/Downloads/Datasets/Capstone_Twitter Modelling Datasets/Disaster Tweets Combined/Preprocessed_tweets.csv")

In [5]:
test_data.head()

,tweet_id,tweet_time,tweet_author,tweet_author_id,tweet_language,tweet_lon,tweet_lat,tweet_text,tweet_url,label
0,591903085670215681',Sat Apr 25 09:54:27 +0000 2015,Faali19,2387302745,en,73.547794,32.392816,Dua's for all those affected by the earthquake...,https://twitter.com/Faali19/status/59190308567...,Sympathy and emotional support
1,591906453306249216',Sat Apr 25 10:07:50 +0000 2015,jonikanerva,13137242,en,86.822297,27.952658,Felt the Kathmandu earthquake here in Dengboch...,https://twitter.com/jonikanerva/status/5919064...,Infrastructure and utilities
2,591907531829616640',Sat Apr 25 10:12:07 +0000 2015,ranjalrajesh,75552756,en,74.839548,12.875823,#NepalEarthquake Tribhuvan Int. Arprt #Kathman...,https://twitter.com/ranjalrajesh/status/591907...,Infrastructure and utilities
3,591907839355957248',Sat Apr 25 10:13:20 +0000 2015,mkaswala,282516228,en,72.638678,23.025132,Gujarat tourist stranded in Nepal/their relati...,https://twitter.com/mkaswala/status/5919078393...,"Missing, trapped, or found people"
4,591907977294143488',Sat Apr 25 10:13:53 +0000 2015,FarrukhFL,2686906843,en,67.023564,24.908860,Pray for #Nepal where a powerful earthquake ha...,https://twitter.com/FarrukhFL/status/591907977...,Sympathy and emotional support


In [6]:
categorizing_model, filtering_model = load_models()
categorizing_model_tokenizer, filtering_model_tokenizer = load_tokenizers()

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
STOPWORDS = stopwords.words('english')

In [8]:
# Dictionary mapping word contractions to their full words
contractions = { 
    "ain't": "are not","'s":" is","aren't": "are not",
     "can't": "cannot","can't've": "cannot have",
     "'cause": "because","could've": "could have","couldn't": "could not",
     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
     "hasn't": "has not","haven't": "have not","he'd": "he would",
     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
     "it'd": "it would","it'd've": "it would have","it'll": "it will",
     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
     "mayn't": "may not","might've": "might have","mightn't": "might not", 
     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
     "mustn't've": "must not have", "needn't": "need not",
     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
     "she'll": "she will", "she'll've": "she will have","should've": "should have",
     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
     "there'd've": "there would have", "they'd": "they would",
     "they'd've": "they would have","they'll": "they will",
     "they'll've": "they will have", "they're": "they are","they've": "they have",
     "to've": "to have","wasn't": "was not","we'd": "we would",
     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
     "what'll've": "what will have","what're": "what are", "what've": "what have",
     "when've": "when have","where'd": "where did", "where've": "where have",
     "who'll": "who will","who'll've": "who will have","who've": "who have",
     "why've": "why have","will've": "will have","won't": "will not",
     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
     "y'all'd've": "you all would have","y'all're": "you all are",
     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
     "you'll": "you will","you'll've": "you will have", "you're": "you are",
     "you've": "you have"
}


# Remove punctutation marks, stopwords, emojis, urls, convert to lowercase, expand contractions, hashtags, retweet
def preprocess_tweet(tweet):
    res_tweet = []
#     lemmatizer = WordNetLemmatizer()
    
    for word in tweet.split():        
        
        # Expand Contractions
        word = contractions.get(word.lower(), word)
            
        # Remove url
        word = re.sub(r'http\S+', '', word)

        # Remove usernames
        word = re.sub('@[\w]+','',word)

        # Remove hashtag
        word = re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", '', word)         # Remove hashtag symbol

        emoji_clean= re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
        word = emoji_clean.sub(r'', word)

        # Remove Retweet
        word = re.sub(r'RT', '', word)

        # Convert to lowercase
        word = word.lower()

        if word != '':
            res_tweet.append(word)


    return ' '.join([word for word in res_tweet]) 

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [10]:
test_data = shuffle(test_data, random_state=0)

In [11]:
test_data = test_data[:200]

In [13]:
# tweet_text = test_data['Tweet'].apply(lambda x: preprocess_tweet(x))
tweet_text = test_data['tweet_text'].apply(lambda x: str(x))

In [14]:
relevance_preds_cat = predict_filter_relevance(filtering_model, filtering_model_tokenizer, tweet_text)

In [15]:
relevance_preds_cat

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1], dtype=int64)

In [16]:
test_data['Relevance'] = relevance_preds_cat

In [17]:
test_data.head(50)

,tweet_id,tweet_time,tweet_author,tweet_author_id,tweet_language,tweet_lon,tweet_lat,tweet_text,tweet_url,label,Relevance
158,592686533423013888',Mon Apr 27 13:47:35 +0000 2015,kellie4brendan,117238554,en,-84.905410,34.383024,Sending prayers to Nepal! #GodBless #NepalQuak...,https://twitter.com/kellie4brendan/status/5926...,Not Relevant,1
83,592589490641743872',Mon Apr 27 07:21:59 +0000 2015,SyahirahSyahira,465192725,en,100.531224,6.438655,@AnizaZulkifle he's fine tho. im glad that u c...,https://twitter.com/SyahirahSyahira/status/592...,Not Relevant,0
170,592703386425896960',Mon Apr 27 14:54:33 +0000 2015,lbhartwick,98297603,en,-76.475897,44.264253,Can't complain about a thing in my life right ...,https://twitter.com/lbhartwick/status/59270338...,Not Relevant,0
101,592592960803479552',Mon Apr 27 07:35:46 +0000 2015,AnggaAdiWijaya,292139731,en,115.089293,-8.108865,"God knows,God listen what's happening in Nepal...",https://twitter.com/AnggaAdiWijaya/status/5925...,Not Relevant,0
150,592664844530262016',Mon Apr 27 12:21:24 +0000 2015,man079,46090945,en,150.900544,-33.870488,@QandA #ArthurSinodinos Fixing buildings in #N...,https://twitter.com/man079/status/592664844530...,Not Relevant,0
199,592765243815788545',Mon Apr 27 19:00:21 +0000 2015,trendinaliaKE,1409839502,en,37.903900,-0.024400,6. #MaishaSuperstar 7. #TransformingKE 8. Thai...,https://twitter.com/trendinaliaKE/status/59276...,Not Relevant,0
118,592609862019780608',Mon Apr 27 08:42:55 +0000 2015,sanjaycsk,289279037,en,77.029323,11.025398,God please save them #NepalEarthquake,https://twitter.com/sanjaycsk/status/592609862...,Not Relevant,0
227,592872827931463681',Tue Apr 28 02:07:51 +0000 2015,beardeddrewster,1206802807,en,-78.693123,43.100449,Forever thankful that I don't live in #Ferguso...,https://twitter.com/beardeddrewster/status/592...,Not Relevant,0
63,592162818247364609',Sun Apr 26 03:06:32 +0000 2015,h_mahendru,1031733936,en,77.337514,28.364839,Salute to Indian army https://t.co/wWWyLPF98q,https://twitter.com/h_mahendru/status/59216281...,Response Efforts,0
135,592634475437236224',Mon Apr 27 10:20:44 +0000 2015,BryanWeaverDC,81921672,en,-77.043442,38.923194,God bless CNN's @drsanjaygupta He stopped rep...,https://twitter.com/BryanWeaverDC/status/59263...,Response Efforts,1


In [18]:
test_data.shape

(200, 11)

In [19]:
predicted_categories = predict_categories(categorizing_model, categorizing_model_tokenizer, tweet_text)

[[7.36990333e-01 3.73733938e-02 2.08104089e-01 1.20524708e-02
  5.47979586e-03]
 [6.82863176e-01 1.84301108e-01 9.73612517e-02 2.98467446e-02
  5.62777603e-03]
 [9.63974476e-01 9.50834434e-03 1.56192761e-02 5.83310798e-03
  5.06472727e-03]
 [9.80431259e-01 2.57778866e-03 1.45127010e-02 1.77845545e-03
  6.99753349e-04]
 [8.59161615e-02 1.47644475e-01 9.29008201e-02 6.68101907e-01
  5.43668773e-03]
 [3.80899466e-04 3.22127459e-03 7.51604373e-03 3.13932658e-03
  9.85742450e-01]
 [8.88055682e-01 1.83676239e-02 7.68796206e-02 1.39983837e-02
  2.69871042e-03]
 [9.71333265e-01 5.08098584e-03 1.43174063e-02 6.45690039e-03
  2.81140837e-03]
 [1.73589170e-01 2.28533894e-02 7.63924956e-01 2.88628452e-02
  1.07697146e-02]
 [8.37470949e-01 1.57496836e-02 8.65876898e-02 2.67802365e-02
  3.34114023e-02]
 [9.37098622e-01 1.30209755e-02 2.37062871e-02 2.45387815e-02
  1.63534307e-03]
 [8.46169829e-01 3.33822556e-02 8.31840858e-02 1.33947367e-02
  2.38690618e-02]
 [4.32182342e-01 5.84394932e-02 4.301988

In [20]:
predicted_categories

array(['Sympathy and Support', 'Sympathy and Support',
       'Sympathy and Support', 'Sympathy and Support', 'Damage Report',
       'Casualty', 'Sympathy and Support', 'Sympathy and Support',
       'Relief Request and Donations', 'Sympathy and Support',
       'Sympathy and Support', 'Sympathy and Support',
       'Sympathy and Support', 'Sympathy and Support', 'Damage Report',
       'Sympathy and Support', 'Relief Request and Donations',
       'Sympathy and Support', 'Sympathy and Support',
       'Sympathy and Support', 'Sympathy and Support',
       'Sympathy and Support', 'Relief Request and Donations',
       'Relief Request and Donations', 'Sympathy and Support',
       'Sympathy and Support', 'Relief Request and Donations',
       'Sympathy and Support', 'Relief Request and Donations',
       'Sympathy and Support', 'Sympathy and Support',
       'Relief Request and Donations', 'Caution and Advice',
       'Sympathy and Support', 'Casualty', 'Sympathy and Support',
       '

In [21]:
test_data['Predicted Category'] = predicted_categories

In [22]:
test_data

,tweet_id,tweet_time,tweet_author,tweet_author_id,tweet_language,tweet_lon,tweet_lat,tweet_text,tweet_url,label,Relevance,Predicted Category
158,592686533423013888',Mon Apr 27 13:47:35 +0000 2015,kellie4brendan,117238554,en,-84.905410,34.383024,Sending prayers to Nepal! #GodBless #NepalQuak...,https://twitter.com/kellie4brendan/status/5926...,Not Relevant,1,Sympathy and Support
83,592589490641743872',Mon Apr 27 07:21:59 +0000 2015,SyahirahSyahira,465192725,en,100.531224,6.438655,@AnizaZulkifle he's fine tho. im glad that u c...,https://twitter.com/SyahirahSyahira/status/592...,Not Relevant,0,Sympathy and Support
170,592703386425896960',Mon Apr 27 14:54:33 +0000 2015,lbhartwick,98297603,en,-76.475897,44.264253,Can't complain about a thing in my life right ...,https://twitter.com/lbhartwick/status/59270338...,Not Relevant,0,Sympathy and Support
101,592592960803479552',Mon Apr 27 07:35:46 +0000 2015,AnggaAdiWijaya,292139731,en,115.089293,-8.108865,"God knows,God listen what's happening in Nepal...",https://twitter.com/AnggaAdiWijaya/status/5925...,Not Relevant,0,Sympathy and Support
150,592664844530262016',Mon Apr 27 12:21:24 +0000 2015,man079,46090945,en,150.900544,-33.870488,@QandA #ArthurSinodinos Fixing buildings in #N...,https://twitter.com/man079/status/592664844530...,Not Relevant,0,Damage Report
...,...,...,...,...,...,...,...,...,...,...,...,...
34,592070284951195648',Sat Apr 25 20:58:50 +0000 2015,zaurav,287755950,en,-77.414951,38.827308,@KatzOnEarth you got it covered. Kathmandu nei...,https://twitter.com/zaurav/status/592070284951...,Not Relevant,1,Damage Report
28,592068989976748033',Sat Apr 25 20:53:41 +0000 2015,BarnesOrlando,40247624,en,36.884369,-1.282659,Pray for Nepal #NepalEarthquake,https://twitter.com/BarnesOrlando/status/59206...,Sympathy and emotional support,0,Sympathy and Support
183,592743087971119104',Mon Apr 27 17:32:19 +0000 2015,NadiaZehra,292467181,en,73.010548,33.670566,Layers of after shook have snatched nights fro...,https://twitter.com/NadiaZehra/status/59274308...,Not Relevant,0,Sympathy and Support
114,592598667179073536',Mon Apr 27 07:58:26 +0000 2015,jasleen_chhabra,2713701522,en,80.894009,26.797255,Golden Temple sending 1 lakh food packages to ...,https://twitter.com/jasleen_chhabra/status/592...,Volunteer or professional services,0,Relief Request and Donations


In [23]:
test_data['Predicted Category'].value_counts()

Sympathy and Support            114
Relief Request and Donations     63
Casualty                         12
Damage Report                     6
Caution and Advice                5
Name: Predicted Category, dtype: int64

In [25]:
test_data['label'].value_counts()

Not Relevant                          135
Sympathy and emotional support         29
Response Efforts                       17
Volunteer or professional services      4
Urgent Needs                            4
Infrastructure Damage                   4
Infrastructure and utilities            2
Money                                   2
Missing, trapped, or found people       1
Personal updates                        1
Injured or dead people                  1
Name: label, dtype: int64